In [1]:
import importlib
import logging
import os

import datasets
import tensorflow as tf

import transformer_document_embedding as tde

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

2023-02-06 17:19:12.785280: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 17:19:12.938275: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 17:19:12.938293: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-06 17:19:12.964711: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-06 17:19:13.553264: W tensorflow/stream_executor/platform/de

In [2]:
tde = importlib.reload(tde)

In [3]:
LOG_DIR = "../results/notebooks/doc2vec_imdb"
task = tde.tasks.IMDBClassification(data_size_limit=100)
model = tde.models.Doc2VecIMDB(log_dir=LOG_DIR, use_dm=False, dbow_kwargs={"epochs": 1})

2023-02-06 17:19:15,318 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow,d400,n5,mc5,s0.001,t12>', 'datetime': '2023-02-06T17:19:15.318647', 'gensim': '4.2.0', 'python': '3.10.8 (main, Nov  1 2022, 14:18:21) [GCC 12.2.0]', 'platform': 'Linux-6.0.11-1-MANJARO-x86_64-with-glibc2.36', 'event': 'created'}
2023-02-06 17:19:15.338274: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-02-06 17:19:15.338303: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: manjaro-pc
2023-02-06 17:19:15.338309: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: manjaro-pc
2023-02-06 17:19:15.338409: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.60.11
2023-02-06 17:19:15.338425: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.60.11
2023-02-06 17:19:15.338430: I tensorflow/st

In [4]:
doc2vec_train = datasets.combine.concatenate_datasets(
    [task.train, task.unsupervised, task.test]
)
doc2vec_train = doc2vec_train.shuffle()
model._doc2vec.train(doc2vec_train)

2023-02-06 17:19:18,608 : WARNING : Found cached dataset imdb (/home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
2023-02-06 17:19:18,624 : WARNING : Loading cached processed dataset at /home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-6c9a401648fa09b0.arrow
2023-02-06 17:19:20,904 : WARNING : Found cached dataset imdb (/home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
2023-02-06 17:19:20,918 : WARNING : Loading cached processed dataset at /home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-9ef2174596950ddb.arrow
2023-02-06 17:19:23,151 : WARNING : Found cached dataset imdb (/home/dburian/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a7068

In [6]:
ds = model._cls_head_dataset(task.train, training=False)

In [7]:
label_ds = task.train.to_tf_dataset(1, columns=["label"]).unbatch()
list(label_ds.take(4).as_numpy_iterator())

[0, 0, 0, 0]

In [8]:
ds = tf.data.Dataset.zip((ds, label_ds))

In [9]:
ds = ds.shuffle(25000)

In [10]:
ds = ds.batch(2)

In [11]:
list(ds.take(4).as_numpy_iterator())

[(array([[[ 0.00383041, -0.00374159,  0.00129179, ...,  0.00068196,
           -0.00339289,  0.00224405],
          [ 0.00115191, -0.00145747,  0.00225265, ...,  0.00255739,
           -0.00455078, -0.00104928]],
  
         [[ 0.00853373, -0.00368737, -0.00121163, ...,  0.00374191,
           -0.00892638,  0.00246742],
          [ 0.01140269, -0.00852688, -0.00101106, ...,  0.00386177,
           -0.01037293,  0.00100791]]], dtype=float32),
  array([0, 0])),
 (array([[[ 7.5155258e-02, -5.2878477e-02, -1.8147372e-03, ...,
            2.6095346e-02, -9.1855139e-02,  1.0356689e-02],
          [ 5.5019073e-03, -6.7298990e-03,  9.7220829e-05, ...,
            8.4743573e-04, -9.0664644e-03,  4.1881960e-04]],
  
         [[ 2.3298461e-02, -1.7067494e-02, -1.1242531e-03, ...,
            5.2597229e-03, -2.7863495e-02,  1.1197915e-03],
          [ 6.8207242e-02, -4.6731599e-02, -4.4107218e-03, ...,
            2.4513312e-02, -8.1739172e-02,  8.0103669e-03]]], dtype=float32),
  array([0, 0])),


In [5]:
ds = model._cls_head_dataset(task.train)

AttributeError: '_UnbatchDataset' object has no attribute 'isidentifier'